In [8]:
import os
import sys
import math
import operator
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from collections import OrderedDict
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import load_model, save_model
os.chdir("D:\\fyp\Testing")

In [28]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x]), np.array(batch_y)         

class CNN_Model:
    
    def __init__(self, directory):
        
        self.path = directory
      #  df = pd.read_csv("occurrences_train.csv",low_memory=False)
        with open("D:\\fyp\Dataset\\save.pkl","rb") as f:
            hd = pkl.load(f)
#         with open("Data/class_encoding.pkl","rb") as f:
#             self.classes = pkl.load(f)
#         with open("Data/order_encoding.pkl","rb") as f:
#             self.orders = pkl.load(f)
#         with open("Data/family_encoding.pkl","rb") as f:
#             self.families = pkl.load(f)
#         with open("Data/genus_encoding.pkl","rb") as f:
#             self.genuses = pkl.load(f)
#         with open("Data/specie_encoding.pkl","rb") as f:
#             self.species = pkl.load(f)

        self.train_pathdata_x = []
        self.train_seq_y = []
        self.test_pathdata_x = []
        self.test_seq_y = []
        for extra in hd.keys():
            for cls in hd[extra].keys():
                for order in hd[extra][cls].keys():
                    for family in hd[extra][cls][order].keys():
                        for genus in hd[extra][cls][order][family].keys():
                            for specie in hd[extra][cls][order][family][genus]:
                                path=self.path+"\\"+cls+"\\"+order+"\\"+family+"\\"+genus+"\\"+specie
                              #  print(path)
                                for im in os.listdir(path):
                                    self.train_pathdata_x.append(path+im)
        
        
# Commenting the code, if what i try fails , run this         
#         for cls in hd.keys():
#             for order in hd[cls].keys():
#                 for family in hd[cls][order].keys():
#                     for genus in hd[cls][order][family].keys():
#                         for specie in hd[cls][order][family][genus]:
#                             for im in os.listdir(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
#                                                  +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
#                                 self.train_pathdata_x.append(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
#                                                              +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
#                                                              +im)
                            
#         for cls in hd.keys():
#             for order in hd[cls].keys():
#                 for family in hd[cls][order].keys():
#                     for genus in hd[cls][order][family].keys():
#                         for specie in hd[cls][order][family][genus]:
#                             for im in os.listdir(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
#                                                  +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
#                                 self.test_pathdata_x.append(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
#                                                              +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
#                                                              +im)
        
        np.random.shuffle(self.train_pathdata_x)
        np.random.shuffle(self.test_pathdata_x)
        
        for p in self.train_pathdata_x:
            print(p)
            y = p.split("\\")
            c = y[3]
            o = y[4]
            f = y[5]
            g = y[6]
            s = y[7]
            self.train_seq_y.append(np.array([[c],[o],[f],[g],[s]]))
            
#         for p in self.test_pathdata_x:
#             y = p.split("/")
#             c = int(y[3])
#             o = int(y[4])
#             f = int(y[5])
#             g = int(y[6])
#             s = int(y[7])
#             self.train_seq_y.append(np.array([[c],[o],[f],[g],[s]]))
            
    def model_create(self):
        
        classifier = Sequential()
        # Step 1 - Convolution
        classifier.add(TimeDistributed(Conv2D(filters=96, kernel_size=(2, 2), activation = 'relu'), input_shape=(5,32,32,33)))
        classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(2, 2), activation = 'relu')))
#         classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), activation = 'relu')))
#         classifier.add(TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), activation = 'relu')))
        #classifier.add(Conv2D(filters=, kernel_size=(2, 2), activation = 'relu'))
        classifier.add(TimeDistributed(MaxPool2D(pool_size = (2, 2))))
        # Step 3 - Flattening
        classifier.add(TimeDistributed(Flatten()))
        # Step 4 - LSTM
        classifier.add(LSTM(100, return_sequences=True))
        # Step 5 - Full connection
        classifier.add(Dense(1, activation = 'relu'))
        # Compiling the CNN
        classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        classifier.summary()
        return classifier
    
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16):        
        try:
            classifier = load_model("D:\\fyp\Dataset\\code\Models\CNN-RNN_1.h5")
        except:
            classifier = self.model_create()
            train_data = ImageDataGenerator(self.train_pathdata_x, self.train_seq_y, batch_size, crop_size)
            history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=True,shuffle=True)
            classifier.save("D:\\fyp\Dataset\\code\Models\CNN-RNN_1.h5")
        train_data = ImageDataGenerator(self.test_pathdata_x, self.test_seq_y, batch_size, crop_size)
        scores = classifier.evaluate_generator(test_data, use_multiprocessing=True)
        print("Loss : ", scores[0])
        print("Accuracy : ", scores[1])

In [29]:
ob = CNN_Model("D:\\fyp\Testing")

D:\fyp\Testing\Liliopsida\Poales\Poaceae\Hordeum\Hordeum murinumpatch_1836.tif
D:\fyp\Testing\Magnoliopsida\Geraniales\Geraniaceae\Geranium\Geranium dissectumpatch_1551.tif
D:\fyp\Testing\Magnoliopsida\Caryophyllales\Caryophyllaceae\Polycarpon\Polycarpon tetraphyllumpatch_2814.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Plantaginaceae\Veronica\Veronica arvensispatch_3777.tif
D:\fyp\Testing\Magnoliopsida\Asterales\Asteraceae\Picris\Picris hieracioidespatch_2592.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Plantaginaceae\Plantago\Plantago mediapatch_2654.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Rostraria\Rostraria cristatapatch_2990.tif
D:\fyp\Testing\Magnoliopsida\Asterales\Asteraceae\Sonchus\Sonchus asperpatch_3353.tif
D:\fyp\Testing\Magnoliopsida\Dipsacales\Caprifoliaceae\Centranthus\Centranthus ruberpatch_815.tif
D:\fyp\Testing\Liliopsida\Asparagales\Orchidaceae\Platanthera\Platanthera bifoliapatch_2665.tif
D:\fyp\Testing\Magnoliopsida\Rosales\Urticaceae\Parietaria\Parietaria judaicap

D:\fyp\Testing\Magnoliopsida\Brassicales\Brassicaceae\Capsella\Capsella bursa-pastorispatch_690.tif
D:\fyp\Testing\Magnoliopsida\Malpighiales\Euphorbiaceae\Mercurialis\Mercurialis annuapatch_2235.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Poa\Poa annuapatch_2745.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Lolium\Lolium perennepatch_2050.tif
D:\fyp\Testing\Magnoliopsida\Ranunculales\Papaveraceae\Fumaria\Fumaria officinalispatch_1449.tif
D:\fyp\Testing\Magnoliopsida\Dipsacales\Caprifoliaceae\Valerianella\Valerianella locustapatch_3761.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Hordeum\Hordeum murinumpatch_1834.tif
D:\fyp\Testing\Magnoliopsida\Ranunculales\Papaveraceae\Fumaria\Fumaria capreolatapatch_1436.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Bromus\Bromus madritensispatch_382.tif
D:\fyp\Testing\Magnoliopsida\Rosales\Moraceae\Ficus\Ficus caricapatch_1366.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Plantaginaceae\Veronica\Veronica persicapatch_3815.tif
D:\fyp\Testing\Magnoliopsid

D:\fyp\Testing\Pinopsida\Pinales\Cupressaceae\Juniperus\Juniperus communispatch_1909.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Plantaginaceae\Plantago\Plantago lanceolatapatch_2610.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Poa\Poa annuapatch_2720.tif
D:\fyp\Testing\Magnoliopsida\Asterales\Asteraceae\Lactuca\Lactuca serriolapatch_1936.tif
D:\fyp\Testing\Magnoliopsida\Malpighiales\Linaceae\Linum\Linum alpinumpatch_2033.tif
D:\fyp\Testing\Magnoliopsida\Rosales\Urticaceae\Parietaria\Parietaria judaicapatch_2526.tif
D:\fyp\Testing\Liliopsida\Lamiales\Plantaginaceae\Plantago\Plantago lanceolatapatch_3971.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Catapodium\Catapodium rigidumpatch_773.tif
D:\fyp\Testing\Magnoliopsida\Brassicales\Brassicaceae\Iberis\Iberis saxatilispatch_1881.tif
D:\fyp\Testing\Magnoliopsida\Asterales\Asteraceae\Sonchus\Sonchus oleraceuspatch_3427.tif
D:\fyp\Testing\Magnoliopsida\Oxalidales\Oxalidaceae\Oxalis\Oxalis corniculatapatch_2430.tif
D:\fyp\Testing\Liliopsida\Asp

D:\fyp\Testing\Magnoliopsida\Brassicales\Brassicaceae\Cochlearia\Cochlearia officinalispatch_969.tif
D:\fyp\Testing\Magnoliopsida\Caryophyllales\Caryophyllaceae\Minuartia\Minuartia mutabilispatch_2258.tif
D:\fyp\Testing\Liliopsida\Asparagales\Orchidaceae\Anacamptis\Anacamptis pyramidalispatch_333.tif
D:\fyp\Testing\Magnoliopsida\Brassicales\Brassicaceae\Capsella\Capsella bursa-pastorispatch_667.tif
D:\fyp\Testing\Magnoliopsida\Rosales\Urticaceae\Parietaria\Parietaria judaicapatch_2536.tif
D:\fyp\Testing\Liliopsida\Asparagales\Orchidaceae\Serapias\Serapias vomeraceapatch_3196.tif
D:\fyp\Testing\Liliopsida\Poales\Poaceae\Poa\Poa annuapatch_2780.tif
D:\fyp\Testing\Magnoliopsida\Malpighiales\Euphorbiaceae\Mercurialis\Mercurialis annuapatch_2208.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Scrophulariaceae\Scrophularia\Scrophularia alpestrispatch_3069.tif
D:\fyp\Testing\Magnoliopsida\Asparagales\Asparagaceae\Anthericum\Anthericum liliagopatch_426.tif
D:\fyp\Testing\Magnoliopsida\Lamiales\Plant

In [ ]:
ob.fit_generator()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 5, 31, 31, 96)     12768     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 30, 30, 256)    98560     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 15, 15, 256)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 57600)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 100)            23080400  
_________________________________________________________________
dense_1 (Dense)              (None, 5, 1)              101       
Total params: 23,191,829
Trainable params: 23,191,829
Non-trainable params: 0
__________________________________________

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\vhari\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\vhari\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\vhari\Anaconda3\lib\site-packages\keras\utils\data_utils.py", line 572, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\vhari\Anaconda3\lib\site-packages\keras\utils\data_utils.py", line 554, in <lambda>
    initargs=(seqs,))
  File "C:\Users\vhari\Anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    context=self.get_context())
  File "C:\Users\vhari\Anaconda3\lib\multiprocessing\pool.py", line 176, in __init__
    self._repopulate_pool()
  File "C:\Users\vhari\Anaconda3\lib\multiprocessing\pool.py", line 241, in _repopulate_pool
    w.start()
  File "C:\Users\vhari\Anaconda3\lib\multiprocessing\process.py", line 112, in start
    self._popen

In [4]:
import pandas as pd
data = pd.read_csv("D:\\fyp\Dataset\occurrences_train.csv",error_bad_lines=False)
print(data.head())

b'Skipping line 7: expected 1 fields, saw 3\nSkipping line 57: expected 1 fields, saw 3\nSkipping line 177: expected 1 fields, saw 3\nSkipping line 300: expected 1 fields, saw 3\nSkipping line 301: expected 1 fields, saw 3\nSkipping line 302: expected 1 fields, saw 3\nSkipping line 303: expected 1 fields, saw 3\nSkipping line 304: expected 1 fields, saw 3\nSkipping line 305: expected 1 fields, saw 3\nSkipping line 306: expected 1 fields, saw 3\nSkipping line 307: expected 1 fields, saw 3\nSkipping line 308: expected 1 fields, saw 3\nSkipping line 309: expected 1 fields, saw 3\nSkipping line 310: expected 1 fields, saw 3\nSkipping line 311: expected 1 fields, saw 3\nSkipping line 312: expected 1 fields, saw 3\nSkipping line 313: expected 1 fields, saw 3\nSkipping line 314: expected 1 fields, saw 3\nSkipping line 315: expected 1 fields, saw 3\nSkipping line 899: expected 1 fields, saw 3\nSkipping line 1148: expected 1 fields, saw 2\nSkipping line 1149: expected 1 fields, saw 3\nSkipping 

  gbifid;"datasetkey";"occurrenceid";"kingdom";"phylum";"class";"order";"family";"genus";"species";"infraspecificepithet";"taxonrank";"scientificname";"countrycode";"publishingorgkey";"Latitude";"Longitude";"coordinateuncertaintyinmeters";"coordinateprecision";"elevation";"elevationaccuracy";"depth";"depthaccuracy";"eventdate";"day";"month";"year";"taxonkey";"specieskey";"basisofrecord";"institutioncode";"collectioncode";"catalognumber";"recordnumber";"identifiedby";"license";"rightsholder";"recordedby";"typestatus";"establishmentmeans";"lastinterpreted";"mediatype";"issue";"espece_retenue_bdtfx";"test";"patch_id";"etp";"chbio_1";"chbio_2";"chbio_3";"chbio_4";"chbio_5";"chbio_6";"chbio_7";"chbio_8";"chbio_9";"chbio_10";"chbio_11";"chbio_12";"chbio_13";"chbio_14";"chbio_15";"chbio_16";"chbio_17";"chbio_18";"chbio_19";"alti";"awc_top";"bs_top";"cec_top";"crusting";"dgh";"dimp";"erodi";"oc_top";"pd_top";"text";"proxi_eau_fast";"clc";"species_glc_id";"patch_dirname"
0  1562872196;"50c9509d